In [1]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Corrected file path (raw string)
file_path = "share-of-adult-men-overweight-or-obese-vs-daily-supply-of-calories (1).csv"


# Load the dataset
df = pd.read_csv(file_path)

# Rename 'Entity' column to 'Country'
df = df.rename(columns={'Entity': 'Country'})

# Clean and prepare the data
df['Obesity Percentage'] = pd.to_numeric(
    df['Overweight or Obese (NCDRisC (2017))'], 
    errors='coerce'
)
df = df.dropna(subset=['Obesity Percentage'])

# Define First World countries
first_world_countries = [
    'United States', 'Canada', 'United Kingdom', 'Germany', 'France', 
    'Italy', 'Japan', 'Australia', 'New Zealand', 'Switzerland', 
    'Sweden', 'Norway', 'Denmark', 'Finland', 'Netherlands', 
    'Belgium', 'Austria', 'Ireland', 'Spain', 'Portugal', 'Greece',
    'Luxembourg', 'Iceland', 'Singapore', 'South Korea', 'Israel'
]

# Get unique years (filter out negative years)
available_years = sorted(df['Year'].unique())
available_years = [yr for yr in available_years if yr > 0]  # Exclude BC years
min_year = min(available_years)
max_year = max(available_years)

# Create widgets
year_slider = widgets.IntSlider(
    value=max_year,
    min=min_year,
    max=max_year,
    step=1,
    description='Select Year:',
    continuous_update=False,
    style={'description_width': 'initial'}
)

first_world_toggle = widgets.Checkbox(
    value=False,
    description='Show First World Countries Only',
    indent=False,
    layout={'width': '300px'}
)

# Create output widget
out = widgets.Output()

def update_top10_countries(selected_year, show_first_world):
    """Update visualization based on selected year and filter"""
    with out:
        out.clear_output(wait=True)
        
        # Filter data for selected year
        year_data = df[df['Year'] == selected_year]
        
        # Apply First World filter if enabled
        if show_first_world:
            year_data = year_data[year_data['Country'].isin(first_world_countries)]
            title_suffix = " (First World Countries)"
        else:
            title_suffix = ""
        
        if year_data.empty:
            print(f"No data available for {selected_year}")
            return
            
        # Get top 10 countries by obesity percentage
        top10 = year_data.nlargest(10, 'Obesity Percentage')
        top10 = top10.sort_values('Obesity Percentage', ascending=False)
        
        # Create horizontal bar chart
        fig = px.bar(top10,
                     x='Obesity Percentage',
                     y='Country',
                     orientation='h',
                     title=f'Top 10 Countries with Highest Obesity Rates ({selected_year}){title_suffix}',
                     labels={'Obesity Percentage': 'Obesity Percentage (%)', 
                             'Country': ''},
                     color='Obesity Percentage',
                     color_continuous_scale='Reds')
        
        # Add value labels to bars
        fig.update_traces(
            texttemplate='%{x:.1f}%', 
            textposition='outside',
            textfont=dict(color='black', size=11),
            marker_line_color='darkred',
            marker_line_width=0.5
        )
        
        # Customize layout
        fig.update_layout(
            height=500,
            plot_bgcolor='white',
            margin=dict(l=120, r=20, t=70, b=40),
            title_font=dict(size=20),
            title_x=0.5,
            showlegend=False,
            yaxis={'categoryorder': 'array', 'categoryarray': top10['Country'].tolist()}
        )
        
        # Format axes
        fig.update_xaxes(
            title='Percentage of Overweight or Obese Adult Men (%)',
            showgrid=True, 
            gridwidth=0.5, 
            gridcolor='LightGrey',
            range=[0, top10['Obesity Percentage'].max() * 1.15]
        )
        fig.update_yaxes(tickfont=dict(size=12))
        
        fig.show()

# Connect widgets to update function
widgets.interactive_output(
    update_top10_countries, 
    {'selected_year': year_slider, 'show_first_world': first_world_toggle}
)

# Display widgets with organized layout
display(widgets.VBox([
    widgets.Label("Obesity Rate Analysis by Country", style={'font_weight': 'bold', 'font_size': '16px'}),
    widgets.HBox([year_slider]),
    widgets.HBox([first_world_toggle]),
    out
]))